In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import panel as pn
import numpy as np
import pyvista as pv
pn.extension('vtk')
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'

# Experiment Manager

In [2]:
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

In [3]:
from torch_points3d.visualization.experiment_manager import ExperimentManager

In [4]:
experiment_manager = ExperimentManager(DIR)

In [5]:
def update_pointcloud(file):
    camera = [ (3.8930294513702393, 2.595715045928955, -2.18937349319458),
 (0.14671478027594276, 0.2490156926214695, -0.07320725917816162),
 (-0.4286445677280426, 0.8776587247848511, 0.21442723274230957)]
    try:
        experiment_manager.load_ply_file(file)
    except:
        print("Could not load corrupted file %s" % file)
        return
    pl1 = pv.Plotter(notebook=True)
    pl1.camera_position =  camera
    point_cloud = pv.PolyData(experiment_manager.current_pointcloud['xyz'])
    point_cloud['l'] = experiment_manager.current_pointcloud['l']
    pl1.add_points(point_cloud)

    pl2 = pv.Plotter(notebook=True)
    pl2.camera_position =  camera
    point_cloud = pv.PolyData(experiment_manager.current_pointcloud['xyz'])
    point_cloud['p'] = experiment_manager.current_pointcloud['p']
    pl2.add_points(point_cloud)
    
    pl3 = pv.Plotter(notebook=True)
    pl3.camera_position =  camera
    point_cloud = pv.PolyData(experiment_manager.current_pointcloud['xyz'])
    point_cloud['e'] = experiment_manager.current_pointcloud['p'] == experiment_manager.current_pointcloud['l']
    pl3.add_points(point_cloud)
     
    pan_left.object = pl1.ren_win
    pan_right.object = pl2.ren_win
    pan_err.object = pl3.ren_win
    
    

def sync_pans(pans):
    ref = pans[0]
    for pan in pans[1:]:
        ref.jslink(pan, camera='camera', bidirectional=True)

In [6]:
selector_model_name = pn.widgets.Select(name='Model name', options=experiment_manager.model_name_wviz)
selector_paths = pn.widgets.Select(name='Available runs', options=experiment_manager.get_model_wviz_paths(selector_model_name.value))
selector_epoch =  pn.widgets.Select(name='Epoch',options = experiment_manager.from_paths_to_epoch(selector_paths.value))
selector_split =  pn.widgets.Select(name='Split',options = experiment_manager.from_epoch_to_split(selector_epoch.value))
selector_file =  pn.widgets.Select(name='Sample',options = experiment_manager.from_split_to_file(selector_split.value))

pl_left = pv.Plotter(notebook=True)
pan_left = pn.panel(pl_left.ren_win, sizing_mode='stretch_height', orientation_widget=True)

pl_right = pv.Plotter(notebook=True)
pan_right = pn.panel(pl_right.ren_win, sizing_mode='stretch_height',)

pl_err = pv.Plotter(notebook=True)
pan_err = pn.panel(pl_err.ren_win, sizing_mode='stretch_height',)


pans = pn.Row(pn.Column(pan_left, "### Ground truth"),
              pn.Column(pan_right, "### Prediction"),
             pn.Column(pan_err, "### Error"))
sync_pans([pan_left, pan_right, pan_err])
col = pn.Column(selector_model_name, selector_paths, selector_epoch, selector_split, selector_file)

dashboard = pn.Column(
    '## Select file to display',
    pn.Row(col, pans)
)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
def update_model_name(event):
    old_value = selector_paths.value
    selector_paths.options = experiment_manager.get_model_wviz_paths(selector_model_name.value)
    if old_value == selector_epoch.value:
        update_epoch(event)
selector_model_name.param.watch(update_model_name, 'value')

def update_model_path(event):
    old_value = selector_epoch.value
    selector_epoch.options = experiment_manager.from_paths_to_epoch(selector_paths.value)
    if old_value == selector_epoch.value:
        update_epoch(event)
selector_paths.param.watch(update_model_path, 'value')

def update_epoch(event):
    old_value = selector_split.value
    selector_split.options = experiment_manager.from_epoch_to_split(selector_epoch.value)
    if old_value == selector_split.value:
        update_split(event)
selector_epoch.param.watch(update_epoch, 'value')

def update_split(event):
    old_value = selector_file.value
    selector_file.options = experiment_manager.from_split_to_file(selector_split.value)
    if old_value == selector_file.value:
        update_file(event)
selector_split.param.watch(update_split, 'value')

def update_file(event):
    update_pointcloud(selector_file.value)
selector_file.param.watch(update_file, 'value')

In [7]:
dashboard

NameError: name 'dashboard' is not defined